In [ ]:
# Step 0: 파일 업로드
from google.colab import files
uploaded = files.upload()  # 예: merged_data.csv 업로드

#  Step 1: 라이브러리 임포트
import pandas as pd
import shap
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from xgboost import XGBClassifier

#  Step 1.5: 한글 폰트 설정 (Colab 환경 한정)
import matplotlib
import matplotlib.font_manager as fm
!apt-get -qq install fonts-nanum
matplotlib.rc('font', family='NanumGothic')
plt.rcParams['axes.unicode_minus'] = False
print(" 한글 폰트 설정 완료")

# Step 2: 데이터 로딩 (인코딩 오류 방지)
file_name = list(uploaded.keys())[0]
df = pd.read_csv(file_name, encoding='utf-8-sig')

# Step 3: 타깃 결측치 제거
df_cleaned = df.dropna(subset=['영남권우울증통계_클러스터'])

#  Step 4: 특성과 타깃 분리
X = df_cleaned.drop(columns=['시군구', '영남권우울증통계_클러스터'])
y = df_cleaned['영남권우울증통계_클러스터'].astype(int)

# Step 5: 결측 열 제거
X = X.dropna(axis=1)

# Step 6: 학습/검증 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

#  Step 7: XGBoost 분류기 모델 학습
model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
model.fit(X_train, y_train)

# Step 8: 성능 평가
y_pred = model.predict(X_test)
print("[분류 성능 요약]")
print(classification_report(y_test, y_pred))

# Step 9: SHAP 값 계산
explainer = shap.Explainer(model)
shap_values = explainer(X_test)

# Step 10: SHAP 요약 플롯 출력
print(" [SHAP Summary Plot]")
shap.summary_plot(shap_values, X_test)  # 한글 폰트 적용됨

# Step 11: SHAP 값 테이블 생성
shap_df = pd.DataFrame(shap_values.values, columns=X_test.columns, index=X_test.index)

# 예측 결과와 시군구 정보 통합
results_df = df_cleaned.iloc[y_test.index].reset_index(drop=True)
combined_df = pd.concat([
    results_df[['시군구']],
    X_test.reset_index(drop=True),
    pd.DataFrame({'예측값': y_pred, '실제값': y_test.reset_index(drop=True)}),
    shap_df.add_prefix('SHAP_')
], axis=1)

# Step 12: CSV 저장 및 다운로드 (한글 인코딩 유지)
output_path = '/content/SHAP_결과_전체.csv'
combined_df.to_csv(output_path, index=False, encoding='utf-8-sig')
files.download(output_path)
